In [1]:
import pickle as pkl
import pandas as pd
from nltk.corpus import stopwords
import nltk
import re
from collections import Counter


In [3]:
#from google.colab import drive
#drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#load data set to df
with open("reviews_data_test1.pkl", "rb") as f:
    object = pkl.load(f)
reviews_df = pd.DataFrame(object)

#load movies data set
with open("movies_data_test1.pkl", "rb") as f:
    object = pkl.load(f)
movies_df = pd.DataFrame(object)


# Process movie data set

In [30]:
movies_df[['year_released','month_released','day_released']] = movies_df['release_date'].str.split('-',expand=True)
dimension_table_1 = movies_df.drop(columns=['Year', 'release_date'])



dimension_table_1
dimension_table_1.to_csv('dimension_table_1_movie_details.csv', index=False)

In [39]:
movies_df.Rating.unique()

array(['R'], dtype=object)

# Process reviews data set

In [8]:
#Expand date data and count number of words
reviews_df[['year_review','month_review','day_review']] = reviews_df['review_date'].str.strip().str[:10].str.split('-',expand=True)
reviews_df['review_no_of_words'] = reviews_df['actual_review'].apply(lambda x: x.count(' ')+1)
reviews_df


,movie_code,IMDb_code,review_date,rating_of_movie,actual_review,year_review,month_review,day_review,review_no_of_words
0,278,tt0111161,2016-04-29T18:08:41.892Z,9.0,very good movie 9.5/10 محمد الشعراوى,2016,04,29,6
1,278,tt0111161,2016-07-10T00:16:50.561Z,10.0,Some birds aren't meant to be caged.\r\n\r\nTh...,2016,07,10,619
2,278,tt0111161,2017-11-11T15:09:34.114Z,6.0,Make way for the best film ever made people. *...,2017,11,11,11
3,278,tt0111161,2018-05-01T05:51:13.756Z,10.0,There is a reason why this movie is at the top...,2018,05,01,62
4,278,tt0111161,2018-10-18T15:08:48.777Z,NaN,It's still puzzling to me why this movie exact...,2018,10,18,91
...,...,...,...,...,...,...,...,...,...
15361,5491,tt0185183,2023-10-08T19:25:04.755Z,6.0,**_The worst sci-fi flick ever made?_**\r\n\r\...,2023,10,08,499
15362,13805,tt1213644,2016-08-15T18:24:29.643Z,1.0,Easily the worst film I have ever seen that wa...,2016,08,15,126
15363,13805,tt1213644,2016-08-15T18:25:43.235Z,1.0,Easily the worst film I have ever seen that wa...,2016,08,15,126
15364,11059,tt0317676,2017-06-21T07:29:00.836Z,4.0,"The dialogue is laughable, the acting is horri...",2017,06,21,52


In [10]:
dimension_table_1.columns

Index(['movie_code', 'Title', 'Year', 'Revenue', 'Budget', 'Runtime',
       'Opening_weekend_US', 'Rating', 'Production_company', 'Genre',
       'IMDb_code', 'release_date', 'year_released', 'month_released',
       'day_released'],
      dtype='object')

In [23]:
#Groupby year,month,day,movie code,imdb code,ratings
#agg by average number of vords, count of reviews

fact_table_1  = reviews_df.groupby(["IMDb_code","movie_code","year_review","month_review","day_review","rating_of_movie"],as_index=False).agg(
    ave_review_len=pd.NamedAgg(column="review_no_of_words", aggfunc="mean"),
    review_count=pd.NamedAgg(column="actual_review", aggfunc="count")
)
selected_dim = dimension_table_1[["Title","IMDb_code","movie_code",'Revenue', 'Budget', 'Runtime','Opening_weekend_US', 'Rating']]

fact_table_1 = pd.merge(fact_table_1, selected_dim, on=["IMDb_code","movie_code"], how='left')
fact_table_1
fact_table_1.to_csv('fact_table_1_movie_stat.csv', index=False)

In [33]:
fact_table_2.dtypes

IMDb_code              object
movie_code             object
reviewer_score         object
review_date_conso      object
actual_review_conso    object
dtype: object

# Remove stopwords

In [24]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gerald\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [25]:
# Remove stopwords from the 'Review' column
stop_words = set(stopwords.words('english'))
reviews_df['actual_review'] = reviews_df['actual_review'].apply(lambda x: ' '.join([word for word in re.sub(r'[^\w\s]', '', str(x)).split() if word.lower() not in stop_words]))
reviews_df

,movie_code,IMDb_code,review_date,rating_of_movie,actual_review,year_review,month_review,day_review,review_no_of_words
0,278,tt0111161,2016-04-29T18:08:41.892Z,9.0,good movie 9510 محمد الشعراوى,2016,04,29,6
1,278,tt0111161,2016-07-10T00:16:50.561Z,10.0,birds arent meant caged Shawshank Redemption w...,2016,07,10,619
2,278,tt0111161,2017-11-11T15:09:34.114Z,6.0,Make way best film ever made people Make way,2017,11,11,11
3,278,tt0111161,2018-05-01T05:51:13.756Z,10.0,reason movie top popular list find strong perf...,2018,05,01,62
4,278,tt0111161,2018-10-18T15:08:48.777Z,NaN,still puzzling movie exactly continues appear ...,2018,10,18,91
...,...,...,...,...,...,...,...,...,...
15361,5491,tt0185183,2023-10-08T19:25:04.755Z,6.0,_The worst scifi flick ever made_ thousand yea...,2023,10,08,499
15362,13805,tt1213644,2016-08-15T18:24:29.643Z,1.0,Easily worst film ever seen wasnt helmed femal...,2016,08,15,126
15363,13805,tt1213644,2016-08-15T18:25:43.235Z,1.0,Easily worst film ever seen wasnt helmed femal...,2016,08,15,126
15364,11059,tt0317676,2017-06-21T07:29:00.836Z,4.0,dialogue laughable acting horrible thing worse...,2017,06,21,52


In [27]:
# concatenate the string


fact_table_2 = reviews_df
fact_table_2= reviews_df.astype(str).groupby(["IMDb_code","movie_code","rating_of_movie"],as_index=False).agg(
                                              lambda x: ' | '.join(x))



# drop duplicate data
fact_table_2 = fact_table_2.drop_duplicates()[['IMDb_code',	'movie_code',	'rating_of_movie',	'review_date',	'actual_review'] ]

fact_table_2= fact_table_2.rename(columns={'rating_of_movie':'reviewer_score',	'review_date':'review_date_conso',	'actual_review':'actual_review_conso'})


# show the dataframe
fact_table_2.to_csv('fact_table_2_review_conso.csv', index=False)
fact_table_2

,IMDb_code,movie_code,reviewer_score,review_date_conso,actual_review_conso
0,tt0000417,775,5.0,2019-08-25T10:43:45.549Z,knew gonna high score letterboxd Yo important ...
1,tt0000417,775,7.0,2021-02-23T04:46:19.141Z | 2022-08-29T12:16:56...,_A Trip Moon_ 1902 initially titled French _Le...
2,tt0000439,5698,6.0,2023-10-28T12:57:35.590Z,Firstly feel honoured first write review prest...
3,tt0000439,5698,7.0,2024-06-30T18:44:19.500Z,opens quite menacing scene Justus Barnes point...
4,tt0004972,618,10.0,2018-03-03T09:27:30.845Z,achieved known man ever achieved watch work li...
...,...,...,...,...,...
12094,tt9866072,615665,7.0,2020-11-06T18:00:08.262Z | 2020-12-17T15:12:29...,Really good watch would watch recommend odd ja...
12095,tt9873892,736769,5.0,2023-07-23T23:47:08.881Z,drug dealer John Boyega predominantly AfricanA...
12096,tt9893250,601666,6.0,2022-03-28T20:34:15.320Z,Could really happen Marla Grayson Rosamund Pik...
12097,tt9893250,601666,8.0,2021-02-15T19:57:00.255Z,enjoy reading SpoilerFree reviews please follo...


In [18]:
fact_table_2.loc[fact_table_2['IMDb_code']=='tt0018455']['actual_review'].values

array(['Interesting much movies done back 1927 relationships men women understood | Wow Friedrich Murnau really pull stops gorgeously complex beautiful story entirely based human character flaws ordinarily might make one squirm None characters actually names George OBrien contentedly married Janet Gaynor living farm young child One day happens upon visiting Margaret Livingstone urban sophisticate soon captivated Conscious duty wife reluctant join big city rather malevolently suggests wife fake boating accident drowns survives Disgracefully agrees couple set trip tries push overboard bottles flees terror city finally catches craves forgiveness Luckily agrees couple spend charming day together experiencing luxuries metropolis offer caught mother storms head home end day Try might though cannot put memory woman behind thoughts take much darker turn story basically tale good vs evil temptation sin always presented clear cut scenario Despite thoroughly odious selfish man didnt hate OBriens 